In [1]:
## Bibliotecas
import yfinance
import plotly_express as px
import plotly.graph_objects as go
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [54]:
# Ativos: WEGE3, PETR4 e CEAB3
lista_ativo = ['WEGE3','PETR4','CEAB3']
# Primeiro montar gráfico de candlestick 
# Depois fazer scraping e pegar informações do Brazil Journal

In [55]:
# Desenvolvimento do Gráfico

# Carregando dados da yfinance
df = yfinance.download(lista_ativo[1] + '.SA',
                       start='2023-01-01',
                       )

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Adj Close'])
                     ])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

#Fonte:https://plotly.com/python/candlestick-charts/

[*********************100%%**********************]  1 of 1 completed


In [28]:
# Scraping
#Link https://braziljournal.com/?s={nome ativo/nome empresa}

# A princípio vamos botar WEG, uma vez que WEGE só retorna uma pesquisa


#Request
url = 'https://braziljournal.com/?s=weg'

r = requests.get(url)

#Scraping dos dados
soup = BeautifulSoup(r.text,'html.parser')


In [75]:
todas_noticiais = soup.find_all(class_='wrap-bj')
lista_titulo = []
lista_link = []
lista_categoria = []
for i in todas_noticiais:
    elementos_h2 = i.find_all('h2', class_='boxarticle-infos-title')[:3]
    elemento_p = i.find_all('p', class_='boxarticle-infos-tag')[:3]
   # Agora você pode trabalhar com os elementos h2 e p encontrados
  
    for elemento_h2, elemento_p in zip(elementos_h2, elemento_p):
  
        
        lista_titulo.append(elemento_h2.a.text.strip())
        lista_link.append(elemento_h2.a['href'])
        lista_categoria.append(elemento_p.a.text.strip())

        # Imprime o texto dentro da tag <a> dentro de 'p'
        print(elemento_p.a.text.strip())
        # Imprime o valor do atributo 'href' da tag <a> dentro de 'p'    
        # Imprime o texto dentro da tag <a> dentro de 'h2'
        print(elemento_h2.a.text.strip())
        # Imprime o valor do atributo 'href' da tag <a> dentro de 'h2'
        print(elemento_h2.a['href'])
        

Estratégia corporativa
WEG faz sua maior aquisição; investe R$ 2 bi para crescer em motores e geradores
https://braziljournal.com/weg-faz-sua-maior-aquisicao-investe-r-2-bi-para-crescer-em-motores-e-geradores/
WEG
WEG faz parcerias de R$ 1,4 bi em autogeração eólica
https://braziljournal.com/weg-faz-parcerias-de-r-14-bi-em-autogeracao-eolica/
Research
WEG leva um (raro) downgrade no Itaú
https://braziljournal.com/weg-leva-um-raro-downgrade-no-itau/


In [76]:
print(f'Categoria:{lista_categoria[0]}')
print(f'Titulo:{lista_titulo[0]}')
print(f'Link:{lista_link[0]}')

Categoria:Estratégia corporativa
Titulo:WEG faz sua maior aquisição; investe R$ 2 bi para crescer em motores e geradores
Link:https://braziljournal.com/weg-faz-sua-maior-aquisicao-investe-r-2-bi-para-crescer-em-motores-e-geradores/


In [81]:
# Criando função de Scraping da Notícias
def scraping_noticia(stock_name,qntd_noticias = 3):
    #Request
    url = f'https://braziljournal.com/?s={stock_name}'

    r = requests.get(url)

    #Scraping dos dados
    soup = BeautifulSoup(r.text,'html.parser')
    
    #Div do html que contém todas as noticias
    todas_noticias = soup.find_all(class_='wrap-bj')

    #Dicionário para armazenar as informações:
    dict_url = {
        'titulo' : [],
        'link' : [],
        'categoria' : []
    }
    
    #Looping para interagir com cada notícia
    for i in todas_noticias:
        
        # Titulo e url    
        elementos_titulo = i.find_all('h2', class_='boxarticle-infos-title')[:qntd_noticias]
        
        # Tópico da noticia 
        elemento_categoria = i.find_all('p', class_='boxarticle-infos-tag')[:qntd_noticias]
    
        # Looping com os elementos para pegar informações de cada noticia
        for elementos_titulo, elemento_categoria in zip(elementos_titulo, elemento_categoria):
            dict_url['titulo'].append(elementos_titulo.a.text.strip())
            dict_url['link'].append(elementos_titulo.a['href'])
            dict_url['categoria'].append(elemento_categoria.a.text.strip())
    return dict_url

In [84]:
resultados = scraping_noticia('C%26A')

In [85]:
resultados

{'titulo': ['C&A surpreende ao dobrar lucro e elevar Ebitda; ações disparam 16%',
  'C&A dispara pós-balanço; controle de caixa será crucial para 2023, diz CEO',
  'A C&A está barata e a estratégia, correta, diz a XP.  Já o macro…'],
 'link': ['https://braziljournal.com/play/c-acoes-disparam-13/',
  'https://braziljournal.com/play/c-controle-de-caixa-sera-crucial-para-2023-diz-ceo/',
  'https://braziljournal.com/a-ca-esta-barata-e-a-estrategia-correta-diz-a-xp-ja-o-macro/'],
 'categoria': ['Varejo & Consumo', 'Varejo & Consumo', 'Research']}

In [86]:

options_stock = ['WEGE3.SA','PETR4.SA','CEAB3.SA']
simbolo_pesquisa = ['WEG','petrobras%20ação','C%26A']


In [87]:
options_stock.index('WEGE3.SA')

0

In [88]:
from scraping import scraping_noticia

In [89]:
stock = 'WEGE3.SA'
pos = options_stock.index(f'{stock}')
palavra_chave = simbolo_pesquisa[pos]
dict_resultado = scraping_noticia(palavra_chave)


In [90]:
dict_resultado

{'titulo': ['WEG faz sua maior aquisição; investe R$ 2 bi para crescer em motores e geradores',
  'WEG faz parcerias de R$ 1,4 bi em autogeração eólica',
  'WEG leva um (raro) downgrade no Itaú'],
 'link': ['https://braziljournal.com/weg-faz-sua-maior-aquisicao-investe-r-2-bi-para-crescer-em-motores-e-geradores/',
  'https://braziljournal.com/weg-faz-parcerias-de-r-14-bi-em-autogeracao-eolica/',
  'https://braziljournal.com/weg-leva-um-raro-downgrade-no-itau/'],
 'categoria': ['Estratégia corporativa', 'WEG', 'Research']}